# <span style="font-width:bold; font-size: 3rem; color:#1EB182;"><img src="../../images/icon102.png" width="38px"></img> **Hopsworks Feature Store** </span><span style="font-width:bold; font-size: 3rem; color:#333;">- Part 03: Training Dataset and Modeling</span>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/logicalclocks/hopsworks-tutorials/blob/master/advanced_tutorials/air_quality/3_training_dataset_and_modeling.ipynb)

<span style="font-width:bold; font-size: 1.4rem;">This notebook explains how to read from a feature group, create a feature view and training dataset within the feature store, train a model and register it in model registry.</span>

## 🗒️ This notebook is divided into the following sections:

1. Fetch Feature Groups.
2. Define Transformation functions.
3. Create Feature Views.
4. Create Training Dataset with training, validation and test splits.
5. Loading the training data.
6. Train the model.
7. Register model in Hopsworks model registry.

![part2](../../images/02_training-dataset.png) 

In [ ]:
#ignore warnings
import warnings
warnings.filterwarnings('ignore')

## <span style="color:#ff5f27;"> 📡 Connecting to Hopsworks Feature Store </span>

In [ ]:
import hopsworks

project = hopsworks.login()

fs = project.get_feature_store() 

In [ ]:
air_quality_fg = fs.get_or_create_feature_group(
    name = 'air_quality_fg',
    version = 1
)

weather_fg = fs.get_or_create_feature_group(
    name = 'weather_fg',
    version = 1
)

In [ ]:
query = air_quality_fg.select_all().join(weather_fg.select_all())

In [ ]:
query.read()

--- 

## <span style="color:#ff5f27;"> 🖍 Feature View Creation and Retrieving </span>

In [ ]:
query = air_quality_fg.select_all().join(weather_fg.select_all())

query_show = query.show(5)
col_names = query_show.columns

query_show

### <span style="color:#ff5f27;">🧑🏻‍🔬 Transformation functions</span>

Hopsworks Feature Store provides functionality to attach transformation functions to training datasets.

Hopsworks Feature Store also comes with built-in transformation functions such as `min_max_scaler`, `standard_scaler`, `robust_scaler` and `label_encoder`.

In [ ]:
[t_func.name for t_func in fs.get_transformation_functions()]

You can retrieve transformation function you need.

To attach transformation function to training dataset provide transformation functions as dict, where key is feature name and value is online transformation function name.

Also training dataset must be created from the Query object. Once attached transformation function will be applied on whenever save, insert and get_serving_vector methods are called on training dataset object.

In [ ]:
category_cols = ['city','conditions']

mapping_transformers = {
    col_name:fs.get_transformation_function(name='label_encoder') 
    for col_name 
    in category_cols
}

`Feature Views` stands between **Feature Groups** and **Training Dataset**. Сombining **Feature Groups** we can create **Feature Views** which store a metadata of our data. Having **Feature Views** we can create **Training Dataset**.

The Feature Views allows schema in form of a query with filters, define a model target feature/label and additional transformation functions.

In order to create Feature View we can use `FeatureStore.get_or_create_feature_view()` method.

You can specify next parameters:

- `name` - name of a feature group.

- `version` - version of a feature group.

- `labels`- our target variable.

- `transformation_functions` - functions to transform our features.

- `query` - query object with data.

In [ ]:
feature_view = fs.get_or_create_feature_view(
    name='air_quality_fv',
    version=1,
    transformation_functions=mapping_transformers,
    query=query
)

---

## <span style="color:#ff5f27;"> 🏋️ Training Dataset Creation</span>

In Hopsworks training data is a query where the projection (set of features) is determined by the parent FeatureView with an optional snapshot on disk of the data returned by the query.

**Training Dataset  may contain splits such as:** 
* Training set - the subset of training data used to train a model.
* Validation set - the subset of training data used to evaluate hparams when training a model
* Test set - the holdout subset of training data used to evaluate a mode

To create training dataset you use `FeatureView.create_training_data()` method.

Here are some importand things:

- It will inherit the name of FeatureView.

- The feature store currently supports the following data formats for
training datasets: **tfrecord, csv, tsv, parquet, avro, orc**.

- You can choose necessary format using **data_format** parameter.

- **start_time** and **end_time** in order to filter dataset in specific time range.

In [ ]:
td_version, td_job = feature_view.create_training_data()

---
## <span style="color:#ff5f27;">🪝 Training Dataset Retrieval</span>

In [ ]:
train_data, _ = feature_view.get_training_data(
    training_dataset_version=td_version
)
train_data.head()

---
## <span style="color:#ff5f27;">🧬 Modeling</span>

In [ ]:
import pandas as pd

from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error

In [ ]:
train_data = train_data.sort_values(by=["date", 'city'], ascending=[False, True]).reset_index(drop=True)
train_data["aqi_next_day"] = train_data.groupby('city')['aqi'].shift(1)

train_data.head(5)

In [ ]:
X = train_data.drop(columns=["date"]).fillna(0)
y = X.pop("aqi_next_day")

In [ ]:
model = GradientBoostingRegressor()
model.fit(X, y)

## <span style='color:#ff5f27'> 👨🏻‍⚖️ Model Validation

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image

import os

In [ ]:
mae = round(mean_absolute_error(y,model.predict(X)),5)
mae

In [ ]:
pred_df = pd.DataFrame({
    'aqi_real': y.iloc[4:10].values,
    'aqi_pred': map(int, model.predict(X.iloc[4:10]))
    }
)
pred_df

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(12,5))

x_ax = range(len(y))
ax = plt.scatter(x_ax, y, s=5, color="blue", label="original")
ax = plt.plot(x_ax, preds, lw=0.8, color="red", label="predicted")

plt.legend()
plt.title('Regression Quality',fontsize=18)

plt.show()

### <span style="color:#ff5f27;"> 💽 Save plots to model registry</span>

In [ ]:
model_dir="assets"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)
    
fig.savefig(model_dir + 'regression_quality.png')

### <span style="color:#ff5f27;"> 📤 Retrieve plots from model registry</span>

In [ ]:
feature_imp = Image.open(model_dir + 'regression_quality.png')

feature_imp.show()

---
## <span style='color:#ff5f27'>🗄 Model Registry</span>

One of the features in Hopsworks is the model registry. This is where you can store different versions of models and compare their performance. Models from the registry can then be served as API endpoints.

In [ ]:
mr = project.get_model_registry()

### <span style="color:#ff5f27;">⚙️ Model Schema</span>

The model needs to be set up with a [Model Schema](https://docs.hopsworks.ai/machine-learning-api/latest/generated/model_schema/), which describes the inputs and outputs for a model.

A Model Schema can be automatically generated from training examples, as shown below.

In [ ]:
from hsml.schema import Schema
from hsml.model_schema import ModelSchema

input_schema = Schema(X)
output_schema = Schema(y)
model_schema = ModelSchema(input_schema=input_schema, output_schema=output_schema)

model_schema.to_dict()

In [ ]:
import joblib

model_dir="air_quality_model"
if os.path.isdir(model_dir) == False:
    os.mkdir(model_dir)
    
model_path = model_dir + 'air_quality_model.pkl'

joblib.dump(model, model_path)

model = mr.python.create_model(
    name="gradient_boost_model",
    metrics={"mean_absolute_error": mae},
    description="Gradient Boost Regressor.",
    input_example=X.sample(),
    model_schema=model_schema
)

model.save(model_path)

---

### <span style="color:#ff5f27;">🥳 <b> Next Steps  </b> </span>
Congratulations you've now completed the Air Quality tutorial for Managed Hopsworks.

Check out our other tutorials on ➡ https://github.com/logicalclocks/hopsworks-tutorials

Or documentation at ➡ https://docs.hopsworks.ai